In [99]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    model_name="gpt-4o", 
    temperature=0.1,
    streaming=True,
    )

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    # memory_key="history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
            You are a helpful AI changing the name of a movie that user has input to three emojis which can be encoded in utf-8 that represent the movie except '👨\u200d👦'.
            If user asks a question, you should answer it in a helpful way. Please stick to the format of the prompt.
        """),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{movie_name}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(movie_name):
    result = chain.invoke({"movie_name": movie_name, "history": load_memory({})})
    memory.save_context(
        {"input": movie_name},
        {"output": result.content},
    )
    return result.content


In [100]:
invoke_chain("대부")


'🔫👔🇮🇹'

In [101]:
invoke_chain("전에 물어본 영화가 뭔지 답해주세요.")


'당신이 전에 물어본 영화는 "대부"입니다.'